In [1]:
import pandas as pd
from redfin import Redfin
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

client = Redfin()

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [2]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]

df = df.drop_duplicates(keep=False, subset=["FullAddress"])

df.head()

C:\Users\kayle\AppData\Local\Temp\ipykernel_24728\1303363462.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
5,38213500050160000,RES,2900,SIDNEY DR,MESQUITE,751502253,38213500050160000,2900 SIDNEY DR MESQUITE TX 75150
9,38139500580020000,RES,1507,RICHARD ST,MESQUITE,751491438,38139500580020000,1507 RICHARD ST MESQUITE TX 75149


In [3]:
addresses = [df['FullAddress']]

In [4]:
# initialize list for successful pulls
houses_list = []

for index, each in enumerate(addresses[0][30000:40000]):

    # reset values
    url = ''
    address = each

    try:
        # attempt to get data from each address
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']        

        house_data = client.above_the_fold(property_id, listing_id=None)

        baths = house_data['payload']['addressSectionInfo']['baths']
        beds = house_data['payload']['addressSectionInfo']['beds']
        sq_ft_lot = house_data['payload']['addressSectionInfo']['lotSize']
        sq_ft_interior = house_data['payload']['addressSectionInfo']['sqFt']['value']
        year_built = house_data['payload']['addressSectionInfo']['yearBuilt']
        lat = house_data['payload']['addressSectionInfo']['latLong']['latitude']
        long = house_data['payload']['addressSectionInfo']['latLong']['longitude']
        price = house_data['payload']['addressSectionInfo']['priceInfo']['amount']
        price_label = house_data['payload']['addressSectionInfo']['priceInfo']['label']
        img = initial_info['payload']['preloadImageUrls']

        # add to house_data_df
        house_data = {
            'property_id': property_id,
            'full_address': address,
            'beds': beds,
            'baths': baths,
            'year_built': year_built,
            'sq_ft_interior': sq_ft_interior,
            'sq_ft_lot': sq_ft_lot,
            'price': price,
            'price_label': price_label,
            'lat': lat,
            'long': long,
            'img_url': img
        }

        # append data to successful list here
        houses_list.append(house_data)

    except Exception as e:
        # anything that doesn't have above info gets thrown as error
        print(f'error: {e} at address: {address}')

    if (index+1) % 100 == 0:
            print(f'Processed {index+1} addresses')

error: 'exactMatch' at address: 704 BARTON AVE GLENN HEIGHTS TX 75154
error: 'baths' at address: 709 BARTON AVE GLENN HEIGHTS TX 75154
error: 'baths' at address: 815 BARTON AVE GLENN HEIGHTS TX 75154
error: 'amount' at address: 705 BARTON AVE GLENN HEIGHTS TX 75154
error: 'exactMatch' at address: 2502 MCKENZIE RD BALCH SPRINGS TX 75181
error: 'baths' at address: 4 LILLIAN AVE GLENN HEIGHTS TX 75154
error: 'baths' at address: 1900 LILLIAN AVE GLENN HEIGHTS TX 75154
error: 'baths' at address: 11804 ASA DR BALCH SPRINGS TX 75180
error: 'amount' at address: 1906 S UHL RD GLENN HEIGHTS TX 75154
error: 'baths' at address: 703 BARTON AVE GLENN HEIGHTS TX 75154
error: 'baths' at address: 431 LAKESIDE RD BALCH SPRINGS TX 75181
error: 'baths' at address: 5317 WILD OAK  BALCH SPRINGS TX 75180
error: 'baths' at address: 5233 ARROWDELL RD BALCH SPRINGS TX 75180
error: 'baths' at address: 5105 ARROWDELL RD BALCH SPRINGS TX 75180
error: 'baths' at address: 5128 THORNDALE RD BALCH SPRINGS TX 75180
err

In [5]:
# create df
house_data_df = pd.DataFrame(
    houses_list
)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url
0,32239397,1814 S UHL RD GLENN HEIGHTS TX 75154,4,2.0,1965,1978,9875,268538,Redfin Estimate,32.555988,-96.841852,[https://maps.googleapis.com/maps/api/streetvi...
1,32091696,800 GODWIN AVE GLENN HEIGHTS TX 75154,3,2.0,2011,1723,43678,351948,Redfin Estimate,32.552196,-96.838596,[https://maps.googleapis.com/maps/api/streetvi...
2,32091706,812 GODWIN AVE GLENN HEIGHTS TX 75154,4,1.0,1965,2142,176000,613259,Redfin Estimate,32.552175,-96.836147,[https://ssl.cdn-redfin.com/photo/90/mbphoto/9...
3,32091714,1902 S UHL RD GLENN HEIGHTS TX 75154,3,2.0,1978,1436,19998,243919,Redfin Estimate,32.554967,-96.841821,[https://maps.googleapis.com/maps/api/streetvi...
4,32239396,708 BARTON AVE GLENN HEIGHTS TX 75154,3,2.0,1995,1768,37710,309639,Redfin Estimate,32.554683,-96.839769,[https://maps.googleapis.com/maps/api/streetvi...


In [6]:
# extract zip code into a new column
house_data_df['zip_code'] = house_data_df['full_address'].str.slice(-5)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url,zip_code
0,32239397,1814 S UHL RD GLENN HEIGHTS TX 75154,4,2.0,1965,1978,9875,268538,Redfin Estimate,32.555988,-96.841852,[https://maps.googleapis.com/maps/api/streetvi...,75154
1,32091696,800 GODWIN AVE GLENN HEIGHTS TX 75154,3,2.0,2011,1723,43678,351948,Redfin Estimate,32.552196,-96.838596,[https://maps.googleapis.com/maps/api/streetvi...,75154
2,32091706,812 GODWIN AVE GLENN HEIGHTS TX 75154,4,1.0,1965,2142,176000,613259,Redfin Estimate,32.552175,-96.836147,[https://ssl.cdn-redfin.com/photo/90/mbphoto/9...,75154
3,32091714,1902 S UHL RD GLENN HEIGHTS TX 75154,3,2.0,1978,1436,19998,243919,Redfin Estimate,32.554967,-96.841821,[https://maps.googleapis.com/maps/api/streetvi...,75154
4,32239396,708 BARTON AVE GLENN HEIGHTS TX 75154,3,2.0,1995,1768,37710,309639,Redfin Estimate,32.554683,-96.839769,[https://maps.googleapis.com/maps/api/streetvi...,75154


In [7]:
house_data_df.value_counts(['price_label'])

price_label    
Redfin Estimate    8060
dtype: int64

In [8]:
house_data_df.dtypes

property_id         int64
full_address       object
beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
price_label        object
lat               float64
long              float64
img_url            object
zip_code           object
dtype: object

In [9]:
house_data_df.shape

(8060, 13)

In [10]:
# export df as csv
house_data_df.to_csv('./redfin_house_data_batch4.csv', index=False)